In [ ]:
import pandas as pd
import os

BASE_PATH = r"C:\Users\a9188\Documents\00. 2024 QMUL\00. Course\Project\00. ACTUAL"
ANNOTATION_DIR = os.path.join(BASE_PATH, "annotations", "Original")

FRAME_ANNOTATION_PATH = os.path.join(ANNOTATION_DIR, "annotation_(framelevel_refinement).xlsx")
SOUND_ANNOTATION_PATH = os.path.join(ANNOTATION_DIR, "annotation(sound_time unit)_ver2.xlsx")

file_1 = pd.read_excel(FRAME_ANNOTATION_PATH)
file_2 = pd.read_excel(SOUND_ANNOTATION_PATH)
video_df = pd.DataFrame(file_1)
sound_df = pd.DataFrame(file_2)

In [2]:
sound_df = sound_df.rename(columns={
    'Start frame' : 'Start frame(sound)',
    'End frame' : 'End frame(sound)',
    'Sound Type' : 'sound_label'
    })

video_df = video_df.rename(columns={
    'Start Frame' : 'Start frame',
    'End Frame' : 'End frame',
    'Violence Type (Sound)' : 'Violence(Sound) Type',
    'Start Time (s)' : 'Start time(s)',
    'End Time (s)' : 'End time(s)'
})

sound_df_refined = sound_df.drop(columns =[
                                 'Filename',
                                 'Duplicated moment with previous',
                                 'Sound Start',
                                 'Sound End',
                                 'Max frame',
                                 'Max Time'])

video_df_refined = video_df.astype(
    {'Video ID': int,
     'Start frame':float,
     'End frame':float}
     )

sound_df_refined = sound_df_refined.astype(
    {'Video ID':int,
     'Start frame(sound)':float,
     'End frame(sound)':float}
     )

combined_df = video_df_refined.copy()

In [3]:
# 1. match the 'Video ID' column in both dataframes
# 2. if  they match, 
#    - append the 'sound_start_frame' and 'sound_end_frame' from sound_df_refined to video_df_refined
#    - append the 'sound_label' from sound_df_refined to video_df_refined as 'Violence(Sound) Type'

# 3. if there are more than one 'sound_start_frame' and 'sound_end_frame' for the same 'Video ID', 
#    - append them as separate columns in video_df_refined
#    - the columns should be soundtype{i}, startframe{i}, endframe{i}, soundtype2, etc. for the sound labels

In [18]:
combined_df.columns

Index(['Video ID', 'Filename', 'Start frame', 'End frame', 'Start time(s)',
       'End time(s)', 'Violence Type (Video)', 'Video Type',
       'Violence Type (Text)', 'Manual Text', 'Memo'],
      dtype='object')

In [19]:
sound_df_refined.columns

Index(['Video ID', 'Title', 'Violence(Sound) Type', 'Start time(s)',
       'End time(s)', 'Start frame(sound)', 'End frame(sound)', 'sound_label',
       'Memos'],
      dtype='object')

In [ ]:
# First Stage
    if s_video_id from sound_df_refined match with video_df_refined:
then 1) check Start frame in combined_df and then

# Second Stage
2) if sound_df_refined 'sound_start_frame' is bigger than combined_df and smaller then 'End frame' in combined_df, 
- 'Violence(Sound) Type' in sound_df_refined value goes to the column called 'Violence(sound) type{i}}' if this column cell is empty.
- 'Sound label' in sound_df_refined value goes to the column called 'Sound type{i}}' if this column cell is empty
- 'Start frame(sound)' in sound_df_refined value goes to the column called 'sound_start_frame{i}' if this column cell is empty
- 'End frame(sound)' in sound_df_refined value goes to the column called 'sound_end_frame{i}' if this column cell is empty

if the cells are all filled already, then next number of the Violence(Sound) Type{i}, Sound type{i}, sound_start_frame, sound_end_frame will be filled.FileLink

# Third Stage
3) if inserted cell that has bigger End frame(sound) in sound_df_refined than the stated End frame in combined_df, then whatever the max number between End frame(sound)(in sound_df_refined) or 'End frame' in combined_df is goes to the sound_end frame(this is to see if the range go over the End frame stated in the combined_df)

In [ ]:
max_columns = 11
for i in range(1, max_columns + 1):
    combined_df[f'Violence(Sound) Type{i}'] = None
    combined_df[f'Sound type{i}'] = None
    combined_df[f'sound_start_frame{i}'] = None
    combined_df[f'sound_end_frame{i}'] = None

combined_df.sort_values(by=['Video ID', 'Start frame'], inplace=True)
combined_df.reset_index(drop=True, inplace=True)

for sound_idx, sound_row in sound_df_refined.iterrows():
    s_video_id = sound_row['Video ID']
    s_type = sound_row['Violence(Sound) Type']
    s_start = sound_row['Start frame(sound)']
    s_end = sound_row['End frame(sound)']
    s_label = sound_row['sound_label']

    matches = combined_df[
        (combined_df['Video ID'] == s_video_id) &
        (combined_df['Start frame'] <= s_start) &
        (combined_df['End frame'] >= s_start)
    ]

    if matches.empty:
        continue

    for match_idx in matches.index:
        for ix in range(1, max_columns + 1):
            if pd.isna(combined_df.at[match_idx, f'Violence(Sound) Type{ix}']):
                combined_df.at[match_idx, f'Violence(Sound) Type{ix}'] = s_type
                combined_df.at[match_idx, f'Sound type{ix}'] = s_label
                combined_df.at[match_idx, f'sound_start_frame{ix}'] = s_start
                fill_end = min(s_end, combined_df.at[match_idx, 'End frame'])
                combined_df.at[match_idx, f'sound_end_frame{ix}'] = fill_end
                break

        if s_end > combined_df.at[match_idx, 'End frame']:
            remaining_start = combined_df.at[match_idx, 'End frame'] + 1

            while remaining_start <= s_end:
                next_matches = combined_df[
                    (combined_df['Video ID'] == s_video_id) &
                    (combined_df['Start frame'] <= remaining_start) &
                    (combined_df['End frame'] >= remaining_start)
                ]

                if next_matches.empty:
                    break

                for next_idx in next_matches.index:
                    segment_end = combined_df.at[next_idx, 'End frame']

                    for next_ix in range(1, max_columns + 1):
                        if pd.isna(combined_df.at[next_idx, f'Violence(Sound) Type{next_ix}']):
                            combined_df.at[next_idx, f'Violence(Sound) Type{next_ix}'] = s_type
                            combined_df.at[next_idx, f'Sound type{next_ix}'] = s_label
                            combined_df.at[next_idx, f'sound_start_frame{next_ix}'] = remaining_start
                            combined_df.at[next_idx, f'sound_end_frame{next_ix}'] = min(s_end, segment_end)
                            remaining_start = segment_end + 1
                            break
                    break  # always move to next row after a fill
        break  # only one initial match per sound_row

output_path = os.path.join(BASE_PATH, "annotations", "combined_sound_frame_annotations.xlsx")
combined_df.to_excel(output_path, index=False)